In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [10]:
DATA_YEAR = 2022
DATA_MONTH = 2

In [11]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{:d}-{:02d}.parquet'.format(
    DATA_YEAR,
    DATA_MONTH,
))

df['ride_id'] = '{year:04d}/{month:02d}_'.format(
    year=DATA_YEAR,
    month=DATA_MONTH,
) + df.index.astype('str')

In [12]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [13]:
y_pred.std()

5.28140357655334

In [15]:
df_result = pd.DataFrame({
    'ride_id': df['ride_id'],
    'predictions': y_pred,
})

In [18]:
!mkdir -p output

df_result.to_parquet(
    'output/predictions.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

!ls -lsh 'output/predictions.parquet'

118792 -rw-r--r--  1 vladimir  staff    57M Jun 25 22:26 output/predictions.parquet


: 